### Creating and Persisting an ML Model

In [191]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [192]:
df.describe()
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [193]:
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

Create a subset of features as an example.

In [194]:
include = ['school', 'higher','Mjob', 'Fjob', 'studytime', 'paid', 'failures', 'G3']
df.drop(columns=df.columns.difference(include), inplace=True)  # only using 3 features

In [195]:
df.info

<bound method DataFrame.info of     school      Mjob      Fjob  studytime  failures paid higher  G3
0       GP   at_home   teacher          2         0   no    yes   6
1       GP   at_home     other          2         0   no    yes   6
2       GP   at_home     other          2         3  yes    yes  10
3       GP    health  services          3         0  yes    yes  15
4       GP     other     other          2         0  yes    yes  10
..     ...       ...       ...        ...       ...  ...    ...  ..
390     MS  services  services          2         2  yes    yes   9
391     MS  services  services          1         0   no    yes  16
392     MS     other     other          1         3   no    yes   7
393     MS  services     other          1         0   no    yes  10
394     MS     other   at_home          1         0   no    yes   9

[395 rows x 8 columns]>

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [196]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [197]:
df.describe()

,studytime,failures,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000
mean,2.035443,0.334177,10.415190,0.184810
std,0.839240,0.743651,4.581443,0.388636
min,1.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,8.000000,0.000000
50%,2.000000,0.000000,11.000000,0.000000
75%,2.000000,0.000000,14.000000,0.000000
max,4.000000,3.000000,20.000000,1.000000


Drop the G3 score

In [198]:
include = ['school', 'higher','Mjob', 'Fjob', 'studytime', 'paid', 'failures', 'qual_student']
df.drop(columns=df.columns.difference(include), inplace=True) 

Import scikit-learn and build a random forest classifer

In [201]:
import sklearn
from sklearn import preprocessing, neighbors, svm
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import train_test_split

dependent_variable = 'qual_student'

# df = pd.concat([pd.get_dummies(df, columns=['Mjob', 'Fjob', 'paid', 'higher', 'school'], drop_first=True), df.drop(['Mjob', 'Fjob', 'paid', 'higher', 'school'], axis=1)], axis=1)
df = pd.get_dummies(df, drop_first=True).astype(float)
y = df[dependent_variable]
X = df[df.columns.difference([dependent_variable])]
# print(X)

# print(df)

# clf = rf(n_estimators = 1000)
# clf.fit(x, y)

#splitting the train and test sets
X_train, X_test, y_train,y_test= train_test_split(X, y, test_size=0.3)

In [204]:
# pred = clf.predict(x)
# sklearn.metrics.f1_score(y, pred, average='binary')

rfc = rf(criterion='gini', 
        n_estimators=1000750,
        max_depth=5,
        min_samples_split=6,
        min_samples_leaf=6,
        max_features='auto',
        oob_score=True,
        random_state=42,
        n_jobs=-1,
        verbose=1)

rfc.fit(X_train, y_train)
print('****Results****')
print("Accuracy: {:.4%}".format(rfc.score(X_test, y_test)))

/root/.local/share/virtualenvs/fall-22-hw4-bill-and-teedy-s-excellent-adv-ObamcBOO/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [ ]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, 'app/handlers/model.pkl')

In [ ]:
query_df = pd.DataFrame({ 'age' : pd.Series(1) ,'health' : pd.Series(15) ,'absences' : pd.Series(10)})

In [ ]:
pred = clf.predict(query_df)

In [ ]:
x

In [ ]:
type(x)